4 Labels I, O, I-V, O-V

In [1]:
pip install NERDA

     |████████████████████████████████| 3.1 MB 6.0 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
     |████████████████████████████████| 3.3 MB 38.8 MB/s 
     |████████████████████████████████| 59 kB 6.7 MB/s 
     |████████████████████████████████| 895 kB 41.5 MB/s 
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=bd7185e60dc9f36074199b6dd1afa21c9e97eecab2d5c0394fc7492f70825363
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built progressbar
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from collections import defaultdict
import itertools
# import WhitespaceTokenizer() method from nltk
from nltk.tokenize import WhitespaceTokenizer
# Create a reference variable for Class WhitespaceTokenizer
tk = WhitespaceTokenizer()

d = defaultdict(list)
d['sentences'] = []
d['tags'] = []
t = defaultdict(list)
t['sentences'] = []
t['tags'] = []
test = defaultdict(list)
test['sentences'] = []
test['tags'] = []


In [3]:
def nerda_format(data):
  ner = defaultdict(list)
  for line1,line2 in itertools.zip_longest(*[data]*2):
    sents = []
    tags = []
    tokens = iter(tk.tokenize(line1))
    current_tag = "O"
    for token in tokens:
      suffix = ""
      if token in line2:
        suffix = "-V" 
                # Good: even if the verb is complex (aka spaced out),suffix can still apply to the entire phrase
                # Bad: false positive -V may occur, if one word that has multiple meanings occurring more than one time in the same sentence 
      try:
        if token == "[":
          current_tag = "I"
          tags.append(current_tag+suffix)
          sents.append(next(tokens))
        elif token == "]":
          current_tag = "O"
        else:
          tags.append(current_tag+suffix)
          sents.append(token)
      except StopIteration:
        pass
    ner['sentences'].append(sents)
    ner['tags'].append(tags)
  return ner

In [5]:
with open('dev.txt') as data:
    d = nerda_format(data)
    #d['sentences'] = ner['sentences']
    #d['tags'] = ner['tags']

with open('train.txt') as data:
    t = nerda_format(data)
  #  t['sentences'] = ner['sentences']
  #  t['tags'] = ner['tags']

with open('test.txt') as data:
    test = nerda_format(data)
  #  test['sentences'] = ner['sentences']
  #  test['tags'] = ner['tags']

#print(len(d['sentences']))
#print (d['sentences'][1])
#print (d['tags'][1])

In [6]:
d_total_O = 0
t_total_O = 0
test_total_O = 0

d_total_I = 0
t_total_I = 0
test_total_I = 0

d_total_B = 0
t_total_B = 0
test_total_B = 0

d_total_BV = 0
t_total_BV = 0
test_total_BV = 0

d_total_OV = 0
t_total_OV = 0
test_total_OV = 0

d_total_IV = 0
t_total_IV = 0
test_total_IV = 0


for elem in d['tags']:
  d_total_O += elem.count('O')
  d_total_I += elem.count('I')
  d_total_B += elem.count('B')
  d_total_IV += elem.count('I-V')
  d_total_BV += elem.count('B-V')
  d_total_OV += elem.count('O-V')

for elem in t['tags']:
  t_total_O += elem.count('O')
  t_total_I += elem.count('I')
  t_total_B += elem.count('B')
  t_total_IV += elem.count('I-V')
  t_total_BV += elem.count('B-V')
  t_total_OV += elem.count('O-V')

for elem in test['tags']:
  test_total_O += elem.count('O')
  test_total_I += elem.count('I')
  test_total_B += elem.count('B')
  test_total_IV += elem.count('I-V')
  test_total_BV += elem.count('B-V')
  test_total_OV += elem.count('O-V')



In [7]:
print ('I' + '=' + str(d_total_I + t_total_I + test_total_I))
print ('O' + '=' + str(d_total_O + t_total_O + test_total_O))
print ('B' + '=' + str(d_total_B + t_total_B + test_total_B))
print ('IV' + '=' + str(d_total_IV + t_total_IV + test_total_IV))
print ('OV' + '=' + str(d_total_OV + t_total_OV + test_total_OV))
print ('BV' + '=' + str(d_total_BV + t_total_BV + test_total_BV))

I=2340
O=1618
B=0
IV=200
OV=315
BV=0


In [8]:
import NERDA
training = t
validation = d

In [9]:
print(training.get('sentences')[0])

['थः', 'नं', 'छम्ह', 'शक्तिशाली', 'खः', 'धयागु', 'आत्मविश्वास', 'ज्वनाः', 'न्ह्याःवःगु', 'खनेदु']


In [10]:
tag_scheme = [
'I',
'I-V',
'O-V'
]

In [17]:
from NERDA.models import NERDA
model = NERDA(
dataset_training = training,
dataset_validation = validation,
tag_scheme = tag_scheme, 
tag_outside = 'O',
transformer = 'bert-base-multilingual-cased',
hyperparameters = {'epochs' : 11,
                  'warmup_steps' : 10,
                  'train_batch_size': 5,
                  'learning_rate': 0.0001})

Device automatically set to: cuda


Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
model.train()


 Epoch 1 / 11


  0%|          | 0/31 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 261 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 23%|██▎       | 7/31 [00:02<00:07,  3.09it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #49 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 26%|██▌       | 8/31 [00:02<00:07,  3.12it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #50 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packag

Train Loss = 1.0122878753369855 Valid Loss = 0.957363098859787

 Epoch 2 / 11


  0%|          | 0/31 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 261 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 23%|██▎       | 7/31 [00:02<00:07,  3.20it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #49 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 26%|██▌       | 8/31 [00:02<00:07,  3.23it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #50 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packag

Train Loss = 0.7714221842827336 Valid Loss = 0.9877205590407053

 Epoch 3 / 11


...

  0%|          | 0/31 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 261 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 23%|██▎       | 7/31 [00:02<00:07,  3.17it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #49 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 26%|██▌       | 8/31 [00:02<00:07,  3.16it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #50 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packag

Train Loss = 0.20480469409977237 Valid Loss = 0.7716607476274172

 Epoch 11 / 11


  0%|          | 0/31 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #6 length 261 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #14 length 152 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 23%|██▎       | 7/31 [00:02<00:07,  3.18it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #48 length 150 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #49 length 130 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
 26%|██▌       | 8/31 [00:02<00:07,  3.18it/s]/usr/local/lib/python3.7/dist-packages/NERDA/preprocessing.py:80: UserWarning: Sentence #50 length 174 exceeds max_len 128 and has been truncated
  warnings.warn(msg)
/usr/local/lib/python3.7/dist-packag

Train Loss = 0.1946938506537868 Valid Loss = 0.8445955614248911


'Model trained successfully'

In [19]:
model.evaluate_performance(test)

,Level,F1-Score,Precision,Recall
0,I,0.929078,0.885135,0.977612
1,I-V,0.620690,0.692308,0.562500
2,O-V,0.476190,0.384615,0.625000
0,AVG_MICRO,0.849858,NaN,NaN
0,AVG_MICRO,0.675319,NaN,NaN


In [20]:
#text= [['अझ', 'वृहद', 'कथं', 'धायेगु', 'खःसा', 'थ्व', 'विज्ञान', 'ब्रह्माण्ड', 'गथे', 'सनाच्वंगु', 'दु', 'धका', 'स्वयेत','याइगु', 'प्रकृतिया', 'साधारण', 'एनालाइसिस', 'ख']]
print(model.predict([test['sentences'][1]]))
print(test['tags'][1])

[['I', 'I', 'I', 'I-V', 'I', 'I', 'I', 'I', 'I', 'I-V', 'I', 'I', 'O-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
['I', 'I', 'I', 'I', 'I', 'I-V', 'I', 'I', 'I-V', 'I-V', 'I', 'I', 'O-V', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O-V', 'O-V', 'O']


In [21]:
## Getting the confusion matrices for every test sentence

from sklearn.metrics import confusion_matrix
y_true = test['tags']
for n in range (len(test['tags'])):
  y_pred = []
  y_pred = model.predict([test['sentences'][n]])[0]
  #print(y_true[n])
  #print(y_pred)
  print(confusion_matrix(y_true[n], y_pred, labels=["I", "O","I-V","O-V"]))

[[ 3  0  1  0]
 [ 7 13  0  0]
 [ 1  0  0  0]
 [ 0  0  1  0]]
[[8 0 1 0]
 [0 8 0 0]
 [2 0 1 0]
 [0 2 0 1]]
[[7 0 0 0]
 [0 0 0 1]
 [2 0 0 0]
 [0 0 0 1]]
[[9 0 0 0]
 [0 7 0 3]
 [0 0 2 0]
 [0 1 0 0]]
[[14  0  0  0]
 [ 0 12  0  2]
 [ 0  0  1  0]
 [ 0  2  0  0]]
[[5 0 0 0]
 [3 0 0 1]
 [1 0 0 0]
 [0 0 0 1]]
[[6 0 0 0]
 [0 5 0 2]
 [0 0 1 0]
 [0 0 0 2]]
[[10  0  0  0]
 [ 0  0  0  1]
 [ 0  0  1  0]
 [ 0  0  0  1]]
[[14  0  0  0]
 [ 0  3  0  1]
 [ 1  0  0  0]
 [ 0  0  0  2]]
[[10  0  0  0]
 [ 0  0  0  1]
 [ 0  0  0  0]
 [ 0  0  0  1]]
[[15  0  1  0]
 [ 0  0  0  1]
 [ 0  0  0  0]
 [ 0  0  0  1]]
[[21  0  0  0]
 [ 0  0  0  0]
 [ 0  0  2  0]
 [ 0  0  0  0]]
[[9 0 0 0]
 [0 2 0 3]
 [0 0 1 0]
 [0 0 0 0]]


In [22]:
# Sum up the 6by6 confusion matrix result in every test sentence
import numpy as np
from sklearn.metrics import confusion_matrix
y_true = test['tags']
d = np.zeros((4,4))
for n in range (len(test['tags'])):
  y_pred = []
  y_pred = model.predict([test['sentences'][n]])[0]
  #print(y_true[n])
  #print(y_pred)
  d += confusion_matrix(y_true[n], y_pred, labels=["I", "O","I-V","O-V"])
print(d)

[[131.   0.   3.   0.]
 [ 10.  50.   0.  16.]
 [  7.   0.   9.   0.]
 [  0.   5.   1.  10.]]
